In [1]:
import hamr2016
import tensorflow as tf
import numpy as np

In [2]:
track = hamr2016.Track.from_file('../data/for_elise_by_beethoven.mid')
matrix = track.to_matrix()

In [3]:
num_midi = 88
placeholder_width = 16
num_filters = [1, 1]
intermediate_results = []

with tf.Graph().as_default() as graph:
    # Create a placeholder with indeterminate first dimension (batch size)
    # and append a single dimension for the convolutional layer
    input_placeholder = tf.placeholder(
        tf.float32, 
        (None, num_midi, placeholder_width, 1),
        'input_placeholder'
    )
    output_placeholder = tf.placeholder(
        tf.float32,
        (None, num_midi),
        'output_placeholder'
    )
    # Determine the previous number of channels
    previous = 1
    previous_output = input_placeholder
    # Apply multiple convolutional layers
    for i, current in enumerate(num_filters):
        with tf.variable_scope('conv2d_{}'.format(i)):
            # Create weights for a convolutional layer
            weights = tf.get_variable(
                'weights', [8, 8, previous, current],
                initializer=tf.truncated_normal_initializer()
            )
            # Apply the filter
            result = tf.nn.conv2d(
                previous_output, weights, [1, 1, 1, 1], 'VALID', name='result'
            )
            # Add a bias
            bias = tf.get_variable(
                'bias',
                initializer=tf.zeros_initializer(current)
            )
            result = tf.nn.bias_add(result, bias)
            # Apply relu
            result = tf.nn.relu(result)
            # Update the filter size and input
            previous_output = result
            previous = current
            intermediate_results.append(result)
            
    with tf.variable_scope('fully_connected'):
        # Flatten the last layer 
        size = np.prod(result.get_shape().as_list()[1:])
        fc_input = tf.reshape(result, (-1, size))
        weights = tf.get_variable(
            'weights', [size, num_midi],
            initializer=tf.truncated_normal_initializer()
        )
        bias = tf.get_variable('bias', initializer=tf.zeros_initializer(num_midi))
        logits = tf.nn.bias_add(tf.matmul(fc_input, weights), bias)

    # Compute the loss
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits, output_placeholder)
    # Add an optimizer
    optimizer = tf.train.AdamOptimizer()
    train = optimizer.minimize(loss)
            
    init_op = tf.initialize_all_variables()

In [4]:
features, output = matrix.next_batch(16, 10)

In [6]:
session = tf.Session(graph=graph)
session.run(init_op)
session.run(loss, {input_placeholder: features[..., None], output_placeholder: output})

array([[  2.32981175e-01,   8.26036835e+00,   2.53831758e-03,
          1.28900969e+00,   5.58655977e-01,   4.01058845e-04,
          7.52123022e+00,   1.10939583e-02,   2.51262093e+00,
          1.27663149e-03,   1.30867176e+01,   1.08371086e+01,
          4.17684346e-01,   7.13594723e+00,   2.98336323e-04,
          1.39706297e+01,   3.98357439e+00,   2.54658833e+01,
          1.22652092e+01,   5.18870068e+00,   0.00000000e+00,
          1.69262858e-04,   9.38663960e+00,   0.00000000e+00,
          0.00000000e+00,   7.96958447e+00,   2.57772114e-02,
          2.35584774e+01,   2.17558537e-03,   2.14761963e+01,
          2.82962807e-04,   1.12220240e+01,   2.77789187e+00,
          2.38418551e-07,   8.34464686e-07,   9.53673862e-07,
          1.19717188e+01,   0.00000000e+00,   1.77179070e+01,
          6.95657921e+00,   0.00000000e+00,   2.31938782e+01,
          9.24145508e+00,   6.86929762e-01,   2.93250559e-05,
          7.74465513e+00,   1.60239105e+01,   2.38418551e-07,
        

In [ ]:
x[]